#### Imports

In [36]:
import os
import re
import skrf as rf
import pandas as pd

#### Paths

In [37]:
script_directory = os.getcwd()#obtem diretório atual
s2ps_path = os.path.join(script_directory,'..','..','oil_data','oil_data') #cria string caminho para todos os .s2p

#### Regex = hell

In [38]:
regex = r'SAW_06_02_2024_OIL_5W30_(\d+)\.S2P'

In [39]:
def read_s2p_vegetal_data(regex, path):
    dfs = []#lista para ocncatenar todos os dicts de cada medição 
    for filename in os.listdir(path):
        match = re.match(regex, filename)
        if match:
            #obtem informação do nome do arquivo
            oil_name = "5W30"
            measurement_number = int(match.group(1))
            
            #carrega arquivo
            path_to_oil = os.path.join(path, filename)
            s2p_file = rf.Network(path_to_oil)
            
            #extrai frequência e parâmetros S do objeto Network
            frequency = s2p_file.f
            s_parameters = s2p_file.s_db
            
            #Dataframe com informações do objeto Network
            individual_oil_dict = {
                'OIL NAME': oil_name,
                'MEASUREMENT NUMBER': measurement_number,
                'FREQUENCY (MHZ)':frequency/1000000,
                'S11 (db)' : s_parameters[:,0,0],
                'S21 (db)' : s_parameters[:,1,0],
                'S12 (db)' : s_parameters[:,0,1],
                'S22 (db)' : s_parameters[:,1,1],
            }
            df = pd.DataFrame.from_dict(individual_oil_dict)
            dfs.append(df)
            
    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

#### Read Dataframes

In [40]:
df_synthetic = read_s2p_vegetal_data(regex, s2ps_path)

In [41]:
df_synthetic.info

<bound method DataFrame.info of       OIL NAME  MEASUREMENT NUMBER  FREQUENCY (MHZ)  S11 (db)  S21 (db)  \
0         5W30                   1       110.000000 -0.952009 -84.93931   
1         5W30                   1       110.002000 -0.953180 -79.43321   
2         5W30                   1       110.004000 -0.950550 -85.23255   
3         5W30                   1       110.006001 -0.949279 -83.22342   
4         5W30                   1       110.008001 -0.951772 -81.08348   
...        ...                 ...              ...       ...       ...   
29995     5W30                   3       129.991999 -1.269503 -70.81112   
29996     5W30                   3       129.993999 -1.270841 -68.71010   
29997     5W30                   3       129.996000 -1.272502 -69.69337   
29998     5W30                   3       129.998000 -1.270890 -68.21421   
29999     5W30                   3       130.000000 -1.270329 -67.68703   

       S12 (db)  S22 (db)  
0     -76.00990 -1.240048  
1     -80.0

In [42]:
#assegura somente óleo de girassol lido
for name in df_synthetic["OIL NAME"]:
    if name != "5W30":
        print("Diferente de 5W30")

In [43]:
#cria .csv caso precise no futuro.
path_save_csv = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs')
if not os.path.exists(path_save_csv):
   os.makedirs(path_save_csv)
df_synthetic.to_csv(path_save_csv + '/5W30_measurements.csv')

In [44]:
def average_measurements(dataframe):
    #agrupa valores por frequência e calcula média do que interessa
    return dataframe.groupby('FREQUENCY (MHZ)', as_index=False)['S21 (db)'].mean()

In [45]:
df_5W30_s21 = average_measurements(df_synthetic)

In [46]:
df_5W30_s21.info


<bound method DataFrame.info of       FREQUENCY (MHZ)   S21 (db)
0          110.000000 -82.069490
1          110.002000 -80.526490
2          110.004000 -80.108923
3          110.006001 -80.267000
4          110.008001 -80.682017
...               ...        ...
9995       129.991999 -71.124490
9996       129.993999 -70.196373
9997       129.996000 -69.513377
9998       129.998000 -68.656497
9999       130.000000 -68.407293

[10000 rows x 2 columns]>

In [47]:
#cria .csv caso precise no futuro.
path_save_csv_processed = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs','processed')
if not os.path.exists(path_save_csv_processed):
   os.makedirs(path_save_csv_processed)
df_synthetic.to_csv(path_save_csv_processed + '/5W30_measurements_processed.csv')